In [1]:
#!/usr/bin/python3
# coding: utf-8
# sapporo

In [33]:
import codecs
from datetime import datetime as dt
import datetime
import math
import sys
from jma_csvdl import save_jma_data, parse_jma_csv
import json
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
from plotly.subplots import make_subplots
import sys
from urllib.request import urlretrieve
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly

In [3]:
if False:
    #filename="kanagawa.geojson"
    filename="aomori.geojson"
    f=codecs.open(filename, "rb", encoding='utf-8')
    j=json.load(f)
    f.close()
    f=codecs.open(filename.replace("geojson", "min.geojson"), "wb", encoding='utf-8')
    f.write(json.dumps(j, ensure_ascii=False, separators=(',', ':')))
    f.close()

In [46]:
class AirRegistance:
    def __init__(self):
        self.R   = 2e-7 # 半径 [m]
        self.L   = self.R * 2 # 直径 [m]
        self.Fd  = None # 空気抵抗力 [N]
        self.eta = None # 空気粘性率 [kg/ms]
        self.rho = None # 空気密度 [kg/m^3]
        self.v   = 0.80 # 球体速度 [m/s]
        self.Re  = None # レイルズ数
        self.Cd  = None # 抗力係数

    def _calc_rho(self, t, P, M):
        a = 0.0034856447
        b = 272.48
        c = 0.378
        d = 100 * 6.1078
        x = (7.5 * t)/(t + 237.3)
        rho = (a*(P/(t+b))) * (1-M*c*((d*10**x)/(P)))
        return rho

    def _calc_Cd(self, Re):
        Cd = 24/Re + \
            (26*(Re/5.0)) / (1+(Re/5.0)**1.52) + \
            (0.411*(Re/263000)**(-7.94)) / (1+(Re/263000)**(-8)) + \
            ((Re**0.80) / 461000)
        return Cd
    
    def _calc_Fd(self):
        Fd = 0.5 * self.Cd * self.rho * math.pi * \
            self.R**2 * self.v**2
        return Fd
    
    def calc(self, t, P, rh):
        K = t + 273.15
        self.eta = 1.487 * 1e-6 * ((K**1.5)/(K+117))
        self.rho = self._calc_rho(t, P, rh/100.0)
        print("eta:{}, rho:{}".format(self.eta, self.rho))
        self.Re = (self.v * self.rho * self.L) / self.eta
        self.Cd = self._calc_Cd(self.Re)
        print("Re:{} Cd:{}".format(self.Re, self.Cd))
        self.Fd = self._calc_Fd()
        print("Fd:{}".format(self.Fd))
        return self.Fd

In [51]:
ar = AirRegistance()
Fd = ar.calc(t=10.0, P=1015.13, rh=100.0)

eta:1.7705695712890723e-05, rho:0.006798868092492209
Re:0.00012287784817252464 Cd:195316.0341799204
Fd:5.3399151370472533e-11


In [52]:
Fd = ar.calc(t=10.0, P=1015.13, rh=80.0)

eta:1.7705695712890723e-05, rho:0.007944321395765928
Re:0.00014357994669445535 Cd:167154.37685990025
Fd:5.339915698999456e-11


In [53]:
Fd = ar.calc(t=10.0, P=1015.13, rh=60.0)

eta:1.7705695712890723e-05, rho:0.009089774699039646
Re:0.00016428204521638608 Cd:146090.33531067494
Fd:5.339916266786944e-11


In [54]:
Fd = ar.calc(t=10.0, P=1015.13, rh=20.0)

eta:1.7705695712890723e-05, rho:0.011380681305587084
Re:0.00020568624226024752 Cd:116682.69097671565
Fd:5.339917417677941e-11


In [55]:
Fd = ar.calc(t=10.0, P=1015.13, rh=0.0)

eta:1.7705695712890723e-05, rho:0.012526134608860803
Re:0.00022638834078217822 Cd:106012.64525356202
Fd:5.3399179999662545e-11


In [56]:
Fd = ar.calc(t=5.0, P=1015.13, rh=20.0)

eta:1.7456918271166893e-05, rho:0.011923476061932422
Re:0.0002185673485176562 Cd:109806.0896684838
Fd:5.2648881898112785e-11
